In [1]:
import pandas as pd
import numpy as np

tenancy_df = pd.read_csv("tenancy.csv", parse_dates=["tenancy_start_date", "tenancy_end_date"])
vulnerability_df = pd.read_csv("vulnerability.csv")
case_df = pd.read_csv("case.csv", parse_dates=["case_create_date"])

## Table of contents

- Point-in-time
- Vulnerable tenants
- Build tenancy-case level table

## Point-in-time 

Given a unit can have multiple tenants, we need to take a point-in-time view of the data. For example, given the following data where the same unit has 3 different tenants each year from 2019-2021:

unit_ref|tenancy_id |tenancy start date|tenancy end date|
---     |---        | ---              | -----------    |
Unit1   |Tenant0    |2019-01-01        |2019-12-31      |
Unit1   |Tenant1    |2020-01-01        |2020-12-31      |
Unit1   |Tenant2    |2021-01-01        |2021-12-31      |

If we take the point-in-time at 2020-06-01, we will exclude Tenant0 and Tenant2 in our analysis and treat this property as occupied by Tenant1.

In [2]:
date = "2020-06-01"

In [3]:
def take_point_in_time_cut(tenancy_df, date):
    """
    A unit can have multiple tenants over time so we take a point-in-time cut.
    """
    # select entries where the required date is between tenancy start date and end date
    start_date_cond = tenancy_df.tenancy_start_date <= date
    # the end date is null if the tenancy has not ended
    end_date_cond = (date <= tenancy_df.tenancy_end_date) | (tenancy_df.tenancy_end_date.isna()) 
    filtered_df = tenancy_df.loc[start_date_cond & end_date_cond]
    return filtered_df

tenancy_df = take_point_in_time_cut(tenancy_df, date)

## Vulnerable tenants

Given a tenant can have multiple vulnerabilities, we insert two columns for `tenancy_df`:

1. `number_vul`: Number of vulnerabilities per tenant 
2. `vul_ind`: An indicator if the tenant has at least one vulnerability (1 for yes, 0 for no).

In [4]:
def add_tenant_vul_ind_col(tenancy_df, vulnerability_df):    
    # calculate number of vulnerabilities per tenant 
    number_vul_df = vulnerability_df.groupby("tenancy_id")["Vulnerability"].count().rename("number_vul").reset_index()
    
    # add vulnerability indicator
    vulnerability_df = number_vul_df.assign(vul_ind=1)
    
    # merge with tenancy
    merged_df = tenancy_df.merge(vulnerability_df, how="left", on="tenancy_id", validate="one_to_one")
    merged_df["number_vul"].fillna(0, inplace=True)
    merged_df["vul_ind"].fillna(0, inplace=True)
    return merged_df    

tenancy_df = add_tenant_vul_ind_col(tenancy_df, vulnerability_df)
print(tenancy_df["vul_ind"].value_counts())
print(tenancy_df["number_vul"].value_counts())

0.0    7227
1.0    2410
Name: vul_ind, dtype: int64
0.0    7227
1.0    1495
2.0     579
3.0     217
4.0      81
5.0      23
6.0       9
7.0       5
9.0       1
Name: number_vul, dtype: int64


## Build tenancy-case level table

We then need to join `tenancy_df` and `case_df` to link the damp & mould issue to the tenants. However, the cases are reported on a unit level, instead of on a tenancy level. 

Thereore, after joining `tenancy_df` and `case_df` on the unit level, the `tenancy_case_df` might contain entries where the tenants are not living in the unit when the case is reported hence we need to exclude these. For example, given the following data where the same unit has reported 4 cases over time:

unit_ref|tenancy_id |tenancy start date|tenancy end date|case_number|case_create_date|
---     |---        | ---              | -----------    |---        |---             |
Unit 1  |Tenant1    |2020-01-01        |2020-12-31      |Case0      |2019-01-01      |
Unit 1  |Tenant1    |2020-01-01        |2020-12-31      |Case1      |2019-06-01      |
Unit 1  |Tenant1    |2020-01-01        |2020-12-31      |Case2      |2020-01-01      |  
Unit 1  |Tenant1    |2020-01-01        |2020-12-31      |Case3      |2020-06-01      |

We will exclude Case0 and Case1 since the cases were reported outside the tenancy period. 

In [5]:
def join_tenancy_case(tenancy_df, case_df):
    """
    Many-to-many join since
    - A unit can have multiple tenants living at the same time.
    - A unit can report multiple issues over time.
    """
    merged_df = tenancy_df.merge(case_df, how="left", on="unit_ref", validate="many_to_many")
    return merged_df

def exclude_invalid_cases(tenancy_case_df):
    # select entries where the case create date is within the tenancy start date and end date
    start_date_cond = tenancy_case_df.tenancy_start_date <= tenancy_case_df.case_create_date
    # the end date is null if the tenancy has not ended
    end_date_cond = (tenancy_case_df.case_create_date <= tenancy_case_df.tenancy_end_date) | (tenancy_case_df.tenancy_end_date.isna())
    
    filtered_df = tenancy_case_df.loc[start_date_cond & end_date_cond]
    
    return filtered_df

In [6]:
tenancy_case_df = join_tenancy_case(tenancy_df, case_df).pipe(exclude_invalid_cases)

tenancy_case_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42544 entries, 0 to 44824
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   unit_ref            42544 non-null  object        
 1   tenancy_id          42544 non-null  object        
 2   tenancy_start_date  42544 non-null  datetime64[ns]
 3   tenancy_end_date    2501 non-null   datetime64[ns]
 4   number_vul          42544 non-null  float64       
 5   vul_ind             42544 non-null  float64       
 6   case_number         42544 non-null  object        
 7   unit_build_year     40970 non-null  float64       
 8   case_status         42544 non-null  object        
 9   case_type           42544 non-null  object        
 10  case_sub_type       20621 non-null  object        
 11  case_create_date    42544 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(3), object(6)
memory usage: 4.2+ MB


In [7]:
tenancy_case_df.to_csv("tenancy_case.csv", index=False)